In [25]:
# Homework 4                                                                                                           
# Center of Mass Position and Velocity                                                                                 

# import modules                                                                                                       
import numpy as np
import astropy.units as u
from ReadFile import Read
import math

In [114]:
class CenterOfMass:

    def __init__(self, filename, ptype):
        # read in the file and particle type                                                                           
        self.time, self.total, self.data = Read(filename)

        #create an array to store indexes of particles of desired Ptype                                                
        self.index = np.where(self.data['type'] == ptype)

        # store the mass, positions, velocities of only the particles of the given type                                
        self.m = self.data['m'][self.index]

        ##### PLACE other particle properties here: x,y,z,vx,vy,vz ##### 
        self.x = self.data['x'][self.index]
        self.y = self.data['y'][self.index]
        self.z = self.data['z'][self.index]
        self.vx = self.data['vx'][self.index]
        self.vy = self.data['vy'][self.index]
        self.vz = self.data['vz'][self.index]

    def total_mass(self):
        #Note: you can add other keyword arguments into the function, but 'self' must be first                         
        return np.sum(self.m)*u.Msun*1e10
    
     ##### PLACE OTHER FUNCTIONS BELOW #####
        
    def COMdefine(self, xComps, yComps, zComps, mComps):
        # Takes the x, y, z, and m components of the particles in a galaxy and returns its center of mass or velocity
        resultX = 0 # Initializes the result variables in x, y, and z
        resultY = 0
        resultZ = 0
        for i in range(len(xComps)):
            resultX += xComps[i] * mComps[i] * u.Msun * 1e10
            resultY += yComps[i] * mComps[i] * u.Msun * 1e10
            resultZ += zComps[i] * mComps[i] * u.Msun * 1e10
        resultX /= (np.sum(mComps) * u.Msun * 1e10)
        resultY /= (np.sum(mComps) * u.Msun * 1e10)
        resultZ /= (np.sum(mComps) * u.Msun * 1e10)
        return resultX, resultY, resultZ # Returns the center of mass components
    
    def COM_P(self, delta):
        XCOM, YCOM, ZCOM = self.COMdefine(self.x, self.y, self.z, self.m) #Our initial guess for the center of mass of the galaxy
        RCOM = math.sqrt(XCOM ** 2 + YCOM ** 2 + ZCOM ** 2) # The magnitude of this position vector
        RNEW = [] # Initializes an array which will be populated with COM-particle vector distance magnitudes
        RMAX = 0 # This variable will describe the largest vector magnitude in RNEW
        for i in range(len(self.x)):
            # This loop populates RNEW and finds RMAX as described
            mag = self.distance(self.x[i], XCOM, self.y[i], YCOM, self.z[i], ZCOM)
            RNEW.append(mag)
            RMAX = mag if mag > RMAX else RMAX
        RMAX /= 2.0
        RCOM2 = 1000
        while abs(RCOM - RCOM2) > delta:
            indices = []
            for i in range(len(RNEW)):
                if RNEW[i] < RMAX:
                    indices.append(i) # This finds the indices within our new, smaller galaxy volume to refine the COM
            XCOM2, YCOM2, ZCOM2 = self.COMdefine(self.x[indices], self.y[indices], self.z[indices], self.m[indices])
            RCOM2 = math.sqrt(XCOM ** 2 + YCOM ** 2 + ZCOM ** 2) # The magnitude of this new COM position vector
            RNEW = []
            for i in range(len(self.x)):
                # This loop populates RNEW
                mag = self.distance(self.x[i], XCOM2, self.y[i], YCOM2, self.z[i], ZCOM2)
                RNEW.append(mag)
            RMAX /= 2.0
        return XCOM2, YCOM2, ZCOM2
    
    def COM_V(self, XCOM, YCOM, ZCOM):
        RNEW = []
        for i in range(len(self.x)):
            # This loop populates RNEW, the distances between each particle and the Galactic center of mass
            mag = self.distance(self.x[i], XCOM, self.y[i], YCOM, self.z[i], ZCOM)
            RNEW.append(mag)
        indices = []
        for i in range(len(RNEW)):
            if RNEW[i] < 15:
                indices.append(i) # This finds the particles within 15 kpc of the galactic COM
        return self.COMdefine(self.vx[indices], self.vy[indices], self.vz[indices], self.m[indices]) #This finds the COM velocity
    
    def distance(self, x1, x2, y1, y2, z1, z2):
        # This function describes the absolute distance between points (x1,y1,z1) and (x2, y2, z2)
        return math.sqrt((x1 - x2) ** 2 + (y1 - y2) ** 2 + (z1 - z2) ** 2)
   

In [115]:
# EXAMPLE OF USING A CLASS                                                                                             
##########################                                                                                             

# Create a Center of mass object for the MW                                                                            
MWCOM = CenterOfMass("MW_000.txt", 2)

In [110]:
# Calculate quantities for MW data                                                                                     
MW_mass = MWCOM.total_mass()
print("MW Disk Mass: ", MW_mass)

MW Disk Mass:  75000000000.00003 solMass


In [116]:
print("Question 1: ")
def returnForHW(num):
    # This function returns the number given in a nice format for the homework
    return "{:.3E}".format(num)
MWXCOM, MWYCOM, MWZCOM = MWCOM.COM_P(1)
print("MW Center of Mass: (", returnForHW(MWXCOM), " kpc, ", returnForHW(MWYCOM), " kpc, ", returnForHW(MWZCOM), " kpc)")
M31COM = CenterOfMass("M31_000.txt", 2)
M33COM = CenterOfMass("M33_000.txt", 2)
M31XCOM, M31YCOM, M31ZCOM = M31COM.COM_P(1)
M33XCOM, M33YCOM, M33ZCOM = M33COM.COM_P(1)
print("M31 Center of Mass: (", returnForHW(M31XCOM), " kpc, ", returnForHW(M31YCOM), " kpc, ", returnForHW(M31ZCOM), " kpc)")
print("M33 Center of Mass: (", returnForHW(M33XCOM), " kpc, ", returnForHW(M33YCOM), " kpc, ", returnForHW(M33ZCOM), " kpc)")

Question 1: 
MW Center of Mass: ( -1.312E+00  kpc,  2.520E+00  kpc,  -1.427E+00  kpc)
M31 Center of Mass: ( -3.777E+02  kpc,  6.114E+02  kpc,  -2.846E+02  kpc)
M33 Center of Mass: ( -4.762E+02  kpc,  4.914E+02  kpc,  -4.124E+02  kpc)


In [112]:
print("Question 2: ")
MWVXCOM, MWVYCOM, MWVZCOM = MWCOM.COM_V(MWXCOM, MWYCOM, MWZCOM) # Finds the Milky Way COM Velocity
M31VXCOM, M31VYCOM, M31VZCOM = M31COM.COM_V(M31XCOM, M31YCOM, M31ZCOM) # Finds the M31 COM Velocity
M33VXCOM, M33VYCOM, M33VZCOM = M33COM.COM_V(M33XCOM, M33YCOM, M33ZCOM) # Finds the M33 COM Velocity
print("The distance between the centers of mass of the Milky Way and M31 is: ", returnForHW(MWCOM.distance(MWXCOM, M31XCOM, MWYCOM, M31YCOM, MWZCOM, M31ZCOM)), " kpc.")
print("Their relative velocities are: ", returnForHW(MWCOM.distance(MWVXCOM, M31VXCOM, MWVYCOM, M31VYCOM, MWVZCOM, M31VZCOM)), " km/s.")

Question 2: 
The distance between the centers of mass of the Milky Way and M31 is:  7.698E+02  kpc.
Their relative velocities are:  1.172E+02  km/s.


In [118]:
print("Question 3: ")
print("The distance between the centers of mass of M33 and M31 is: ", returnForHW(MWCOM.distance(M33XCOM, M31XCOM, M33YCOM, M31YCOM, M33ZCOM, M31ZCOM)), "kpc.")
print("Their relative velocities are: ", MWCOM.distance(M33VXCOM, M31VXCOM, M33VYCOM, M31VYCOM, M33VZCOM, M31VZCOM), " km/s.")

Question 3: 
The distance between the centers of mass of M33 and M31 is:  2.011E+02 kpc.
Their relative velocities are:  199.36249797882314  km/s.


Question 4:
M31 and the Milky Way merging will frequently cause ejections of particles from the system, among other issues. Particles that may be considered "part" of one or the other galaxy in the simulation may not be representative of the galactic dynamics, in ways that central particles will. Therefore, iteratively determining the galactic COM using more central particles (as we have done) provides a better representation of the galaxies' dynamics